In [29]:
import pandas as pd
import numpy as np


In [30]:
df = pd.read_csv ("tmdb.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            5000 non-null   int64  
 1   title         5000 non-null   object 
 2   release_date  5000 non-null   object 
 3   vote_average  5000 non-null   float64
 4   genre_names   5000 non-null   object 
 5   duration      5000 non-null   int64  
 6   description   5000 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 273.6+ KB


,id,title,release_date,vote_average,genre_names,duration,description
0,1156593,Your Fault,2024-12-26,7.271,"['Romance', 'Drama']",120,The love between Noah and Nick seems unwaverin...
1,558449,Gladiator II,2024-11-05,6.775,"['Action', 'Adventure', 'Drama']",148,Years after witnessing the death of the revere...
2,845781,Red One,2024-10-31,7.032,"['Action', 'Fantasy', 'Comedy']",124,After Santa Claus (codename: Red One) is kidna...
3,939243,Sonic the Hedgehog 3,2024-12-19,7.700,"['Action', 'Science Fiction', 'Comedy', 'Family']",110,"Sonic, Knuckles, and Tails reunite against a p..."
4,912649,Venom: The Last Dance,2024-10-22,6.803,"['Action', 'Science Fiction', 'Adventure', 'Th...",109,Eddie and Venom are on the run. Hunted by both...


In [31]:
df.isnull().sum()

id              0
title           0
release_date    0
vote_average    0
genre_names     0
duration        0
description     0
dtype: int64

In [32]:
# Konversi ke tipe data yang sesuai
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
# Membersihkan teks
df['description'] = df['description'].str.strip().str.lower()

In [33]:
# Tampilkan contoh data genre_names
print(df['genre_names'].head())
print(df['genre_names'].iloc[0])  # Detail elemen pertama
print(type(df['genre_names'].iloc[0]))  # Tipe data elemen pertama


0                                 ['Romance', 'Drama']
1                     ['Action', 'Adventure', 'Drama']
2                      ['Action', 'Fantasy', 'Comedy']
3    ['Action', 'Science Fiction', 'Comedy', 'Family']
4    ['Action', 'Science Fiction', 'Adventure', 'Th...
Name: genre_names, dtype: object
['Romance', 'Drama']
<class 'str'>


In [34]:
import ast

# Ubah string representasi list menjadi list Python
df['genre_names'] = df['genre_names'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [35]:
print(df['genre_names'].head())
print(df['genre_names'].iloc[0])  # Detail elemen pertama
print(type(df['genre_names'].iloc[0]))  # Tipe data elemen pertama


0                                  [Romance, Drama]
1                        [Action, Adventure, Drama]
2                         [Action, Fantasy, Comedy]
3         [Action, Science Fiction, Comedy, Family]
4    [Action, Science Fiction, Adventure, Thriller]
Name: genre_names, dtype: object
['Romance', 'Drama']
<class 'list'>


In [36]:
df.head()

,id,title,release_date,vote_average,genre_names,duration,description
0,1156593,Your Fault,2024-12-26,7.271,"[Romance, Drama]",120,the love between noah and nick seems unwaverin...
1,558449,Gladiator II,2024-11-05,6.775,"[Action, Adventure, Drama]",148,years after witnessing the death of the revere...
2,845781,Red One,2024-10-31,7.032,"[Action, Fantasy, Comedy]",124,after santa claus (codename: red one) is kidna...
3,939243,Sonic the Hedgehog 3,2024-12-19,7.700,"[Action, Science Fiction, Comedy, Family]",110,"sonic, knuckles, and tails reunite against a p..."
4,912649,Venom: The Last Dance,2024-10-22,6.803,"[Action, Science Fiction, Adventure, Thriller]",109,eddie and venom are on the run. hunted by both...


In [37]:
from mlxtend.preprocessing import TransactionEncoder
# One-Hot Encoding untuk genre
transactions = df['genre_names'].tolist()
te = TransactionEncoder()
te_data = te.fit(transactions).transform(transactions)
genre_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Gabungkan dengan DataFrame utama
df = pd.concat([df, genre_encoded], axis=1)

print(genre_encoded.head())

   Action  Adventure  Animation  Comedy  Crime  Drama  Family  Fantasy  \
0   False      False      False   False  False   True   False    False   
1    True       True      False   False  False   True   False    False   
2    True      False      False    True  False  False   False     True   
3    True      False      False    True  False  False    True    False   
4    True       True      False   False  False  False   False    False   

   Horror  Music  Romance  Science Fiction  Thriller  
0   False  False     True            False     False  
1   False  False    False            False     False  
2   False  False    False            False     False  
3   False  False    False             True     False  
4   False  False    False             True      True  


In [38]:
from mlxtend.frequent_patterns import apriori

# Menemukan frequent itemsets
frequent_itemsets = apriori(genre_encoded, min_support=0.01, use_colnames=True)
print(frequent_itemsets)

    support                                        itemsets
0      0.65                                        (Action)
1      0.30                                     (Adventure)
2      0.20                                     (Animation)
3      0.20                                        (Comedy)
4      0.10                                         (Crime)
..      ...                                             ...
76     0.05  (Action, Adventure, Thriller, Science Fiction)
77     0.05    (Action, Family, Science Fiction, Animation)
78     0.05       (Comedy, Action, Family, Science Fiction)
79     0.05          (Comedy, Family, Adventure, Animation)
80     0.05           (Family, Adventure, Music, Animation)

[81 rows x 2 columns]


In [39]:
from mlxtend.frequent_patterns import association_rules

# Menemukan association rules dengan lift sebagai metrik
rules = association_rules(frequent_itemsets, metric='confidence',min_threshold=0.1)

# Menampilkan aturan asosiasi
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


            antecedents                     consequents  support  confidence  \
0              (Action)                     (Adventure)     0.20    0.307692   
1           (Adventure)                        (Action)     0.20    0.666667   
2              (Action)                     (Animation)     0.10    0.153846   
3           (Animation)                        (Action)     0.10    0.500000   
4              (Comedy)                        (Action)     0.15    0.750000   
..                  ...                             ...      ...         ...   
297  (Music, Animation)             (Adventure, Family)     0.05    1.000000   
298            (Family)   (Adventure, Music, Animation)     0.05    0.250000   
299         (Adventure)      (Family, Music, Animation)     0.05    0.166667   
300             (Music)  (Adventure, Family, Animation)     0.05    1.000000   
301         (Animation)      (Adventure, Family, Music)     0.05    0.250000   

          lift  
0     1.025641  
1    

In [40]:
from sklearn.preprocessing import MultiLabelBinarizer

# MultiLabelBinarizer untuk one-hot encode genre
mlb = MultiLabelBinarizer()

# One-hot encoding genre_names
genre_encoded = mlb.fit_transform(df['genre_names'])

# Mengubah menjadi DataFrame dengan nama kolom yang sesuai
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

# Gabungkan dengan data asli
df = pd.concat([df, genre_df], axis=1)

# Menampilkan DataFrame yang sudah memiliki kolom genre terencode
print(df.head())


        id                  title release_date  vote_average  \
0  1156593             Your Fault   2024-12-26         7.271   
1   558449           Gladiator II   2024-11-05         6.775   
2   845781                Red One   2024-10-31         7.032   
3   939243   Sonic the Hedgehog 3   2024-12-19         7.700   
4   912649  Venom: The Last Dance   2024-10-22         6.803   

                                      genre_names  duration  \
0                                [Romance, Drama]       120   
1                      [Action, Adventure, Drama]       148   
2                       [Action, Fantasy, Comedy]       124   
3       [Action, Science Fiction, Comedy, Family]       110   
4  [Action, Science Fiction, Adventure, Thriller]       109   

                                         description  Action  Adventure  \
0  the love between noah and nick seems unwaverin...   False      False   
1  years after witnessing the death of the revere...    True       True   
2  after sa

In [41]:
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur dan target
x = df[mlb.classes_]  # Fitur berdasarkan genre
y = df['vote_average']  # Target (rating)

# Bagi data menjadi training dan testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Menampilkan bentuk data
print(f"X_train shape: {x_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (4000, 26)
X_test shape: (1000, 26)
y_train shape: (4000,)
y_test shape: (1000,)


In [42]:
# Menyusun ulang X_train dan X_test agar memiliki kolom yang sama
x_train, x_test = x_train.align(x_test, join='outer', axis=1, fill_value=0)

# Menampilkan bentuk data setelah align
print(f"Setelah align - X_train shape: {x_train.shape}")
print(f"Setelah align - X_test shape: {x_test.shape}")

Setelah align - X_train shape: (4000, 26)
Setelah align - X_test shape: (1000, 26)


In [43]:
from sklearn.ensemble import RandomForestRegressor

# Inisialisasi model RandomForest
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Latih model dengan data training
model.fit(x_train, y_train)

# Prediksi rating menggunakan data testing
y_pred = model.predict(x_test)

# Evaluasi model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Menghitung evaluasi
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R2: {r2}")


MAE: 0.039313518751274215
MSE: 0.010090918681404258
R2: 0.9769398883590507


In [44]:
def predict_rating(new_genre, mlb, model, x_train):
    """
    Memprediksi rating berdasarkan genre baru.
    """
    # Validasi genre baru
    invalid_genres = [genre for genre in new_genre if genre not in mlb.classes_]
    if invalid_genres:
        raise ValueError(f"Genre tidak valid: {', '.join(invalid_genres)}")

    # One-hot encoding genre baru
    new_genre_encoded = pd.DataFrame(mlb.transform([new_genre]), columns=mlb.classes_)

    # Reindex agar cocok dengan X_train
    new_genre_encoded = new_genre_encoded.reindex(columns=x_train.columns, fill_value=0)

    # Prediksi rating
    predicted_rating = model.predict(new_genre_encoded)
    return predicted_rating[0]
    
new_genre = ['Action', 'Adventure', 'Drama','Crime']
try:
    rating = predict_rating(new_genre, mlb, model, x_train)
    print(f"Predicted Rating: {rating}")
except ValueError as e:
    print(e)


Predicted Rating: 6.774999999999989


In [45]:
def recommend_movies_with_association(selected_genres, df, rules, mlb, top_n=5):
    """
    Memberikan rekomendasi film berdasarkan genre yang dimasukkan,
    termasuk mempertimbangkan genre tambahan dari aturan asosiasi.

    Parameters:
    - selected_genres (list): Genre yang dimasukkan pengguna.
    - df (pd.DataFrame): DataFrame utama yang memuat informasi film.
    - rules (pd.DataFrame): DataFrame aturan asosiasi.
    - mlb (MultiLabelBinarizer): MultiLabelBinarizer untuk genre.
    - top_n (int): Jumlah rekomendasi yang diinginkan.

    Returns:
    - pd.DataFrame: DataFrame dengan informasi rekomendasi film.
    """
    # Validasi genre
    invalid_genres = [genre for genre in selected_genres if genre not in mlb.classes_]
    if invalid_genres:
        raise ValueError(f"Genre tidak valid: {', '.join(invalid_genres)}")

    # Temukan genre tambahan berdasarkan aturan asosiasi
    additional_genres = set()
    for genre in selected_genres:
        # Temukan aturan asosiasi dengan genre sebagai antecedent
        matched_rules = rules[rules['antecedents'].apply(lambda x: genre in x)]
        # Tambahkan semua consequents dari aturan yang cocok
        for consequents in matched_rules['consequents']:
            additional_genres.update(consequents)

    # Gabungkan genre asli dengan genre tambahan
    all_genres = set(selected_genres).union(additional_genres)

    # Filter film yang memiliki semua genre dalam 'all_genres'
    mask = df['genre_names'].apply(lambda x: all(genre in x for genre in all_genres))
    filtered_movies = df[mask]

    # Urutkan berdasarkan rating tertinggi
    recommended_movies = filtered_movies.sort_values(by='vote_average', ascending=False)

    # Pilih hanya kolom yang relevan
    return recommended_movies[['title', 'release_date', 'vote_average', 'duration', 'description']].head(top_n)


In [46]:
import pickle

# Simpan kolom X_train
filename = 'x_train_columns1.sav'
pickle.dump(x_train.columns, open(filename, 'wb'))
print("X_train columns saved successfully")

filename = 'encorder1.sav'
pickle.dump(mlb, open(filename,'wb'))
print ("encorder save successflly")

filename = 'ekplorasi1.sav'
pickle.dump(rules, open(filename,'wb'))
print ("ekplorasi save successflly")

filename = 'prediksi_model1.sav'
pickle.dump(model, open(filename,'wb'))
print ("Model save successflly")


X_train columns saved successfully
encorder save successflly
ekplorasi save successflly
Model save successflly
